## model

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

def build_model(num_classes):
    # construct CNN structure
    model = Sequential()

    # 1st convolution layer
    model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))

    # 2nd convolution layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

    # 3rd convolution layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    # model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

    model.add(Flatten())

    # fully connected neural networks
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(num_classes, activation='softmax'))

    return model


Using TensorFlow backend.


## train

In [2]:
import numpy as np
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
#from model import build_model
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd


def read_data_np(path):
    with open(path) as f:
        content = f.readlines()

    lines = np.array(content)

    num_of_instances = lines.size
    print("number of instances: ", num_of_instances)
    print("instance length: ", len(lines[1].split(",")[1].split(" ")))

    return lines, num_of_instances


def read_data_pd(path):

    data_df = pd.read_csv(path, header=0)
    lines = len(data_df)
    print(data_df.head())

    return data_df, lines


def reshape_dataset(paths, num_classes):
    x_train, y_train, x_test, y_test = [], [], [], []

    lines, num_of_instances = read_data_np(paths)
    # data_df, lines = read_data_pd(paths)

    # ------------------------------
    # transfer train and test set data
    for i in range(1, num_of_instances):
        try:
            emotion, img, usage = lines[i].split(",")
            # emotion, img, usage = data_df['emotion'][i], data_df['pixels'][i], data_df['Usage'][i]

            val = img.split(" ")

            pixels = np.array(val, 'float32')

            emotion = keras.utils.to_categorical(emotion, num_classes)

            if 'Training' in usage:
                y_train.append(emotion)
                x_train.append(pixels)
            elif 'PublicTest' in usage:
                y_test.append(emotion)
                x_test.append(pixels)
        except:
            print("", end="")

    # ------------------------------
    # data transformation for train and test sets
    x_train = np.array(x_train, 'float32')
    y_train = np.array(y_train, 'float32')
    x_test = np.array(x_test, 'float32')
    y_test = np.array(y_test, 'float32')

    x_train /= 255  # normalize inputs between [0, 1]
    x_test /= 255

    x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
    x_test = x_test.astype('float32')

    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    y_train = y_train.reshape(y_train.shape[0], 7)
    y_train = y_train.astype('int16')
    y_test = y_test.reshape(y_test.shape[0], 7)
    y_test = y_test.astype('int16')

    print('--------x_train.shape:', x_train.shape)
    print('--------y_train.shape:', y_train.shape)


    print(len(x_train), 'train x size')
    print(len(y_train), 'train y size')
    print(len(x_test), 'test x size')
    print(len(y_test), 'test y size')

    return x_train, y_train, x_test, y_test


def batch_process(path, batch_size):
    x_train, y_train, x_test, y_test = reshape_dataset(path)
    gen = ImageDataGenerator()
    train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

    return x_train, y_train, x_test, y_test, train_generator


def compile_model(models):
    model = models.compile(loss='categorical_crossentropy'
                  , optimizer=keras.optimizers.Adam()
                   , metrics=['accuracy']
                   )
    return model


def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))

    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    plt.show()
    return plt


num_classes = 7  # angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5

#batch_size = 128
#num_classes = len(classes)
#epochs =  10

config = tf.ConfigProto(device_count={'GPU': 0, 'CPU': 56})  # max: 1 gpu, 56 cpu
sess = tf.Session(config=config)
keras.backend.set_session(sess)

path = 'fer2013.csv'
x_train, y_train, x_test, y_test = reshape_dataset(path, num_classes)

gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)


    # for data_batch, label_batch in train_generator:
    #     print('data_batch:', data_batch.shape)
    #     print('label_batch:', label_batch.shape)

m = build_model(num_classes)
print('model:', m)
print('train_generator:', train_generator)
# m = compile_model(m)
m.compile(loss='categorical_crossentropy'
               , optimizer=keras.optimizers.Adam()
               , metrics=['accuracy']
               )
print('m:', m)
m.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs)

m.save('facial_expression_model_weights.h5')
print('save weight..')


number of instances:  35888
instance length:  2304
28709 train samples
3589 test samples
--------x_train.shape: (28709, 48, 48, 1)
--------y_train.shape: (28709, 7)
28709 train x size
28709 train y size
3589 test x size
3589 test y size
Instructions for updating:
Colocations handled automatically by placer.
model: <keras.engine.sequential.Sequential object at 0x0000022358F0DF88>
train_generator: <keras.preprocessing.image.NumpyArrayIterator object at 0x000002234E1DD788>
m: <keras.engine.sequential.Sequential object at 0x0000022358F0DF88>
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
256/256 [==============================] - 398s 2s/step - loss: 1.7682 - accuracy: 0.2705
Epoch 2/5
256/256 [==============================] - 401s 2s/step - loss: 1.5586 - accuracy: 0.3875
Epoch 3/5
256/256 [==============================] - 400s 2s/step - loss: 1.3887 - accuracy: 0.4678
Epoch 4/5
256/256 [==============================] - 416s 2s/step - loss: 1.2902 - accuracy: 0.5072
Epoch 5/

## test

In [ ]:
#from train import emotion_analysis, reshape_dataset
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
#from model import build_model

if __name__ == '__main__':
    # path = '/home/jing/PycharmProjects/facial/dataset/fer2013/fer2013.csv'
    num_classes = 7

    # x_train, y_train, x_test, y_test = reshape_dataset(path, num_classes)

    model = build_model(num_classes)
    model.load_weights('facial_expression_model_weights.h5')

    # monitor_testset_results = False
    #
    # if monitor_testset_results == True:
    #     # make predictions for test set
    #     predictions = model.predict(x_test)
    #
    #     index = 0
    #     for i in predictions:
    #         if index < 30 and index >= 20:
    #             # print(i) #predicted scores
    #             # print(y_test[index]) #actual scores
    #
    #             testing_img = np.array(x_test[index], 'float32')
    #             testing_img = testing_img.reshape([48, 48])
    #
    #             plt.gray()
    #             plt.imshow(testing_img)
    #             plt.show()
    #
    #             print(i)
    #
    #             emotion_analysis(i)
    #             print("----------------------------------------------")
    #         index = index + 1

    # ------------------------------
    # make prediction for custom image out of test set

    # img = image.load_img("/home/jing/PycharmProjects/facial/dataset/pablo.png", grayscale=True, target_size=(48, 48))
    # img = image.load_img("/home/jing/PycharmProjects/facial/dataset/monalisa.png", grayscale=True, target_size=(48, 48))
   

In [ ]:
img = image.load_img("xindi.jpg", grayscale=True, target_size=(48, 48))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255
custom = model.predict(x)
t1 = emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48])
plt.gray()

plt.imshow(x)
plt.show()

In [ ]:
fer_json = model.to_json()
with open("fer.json","w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")